In [180]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [183]:
# Loding libray and settingp config 

# Connecting with CCF netowrks.

# Loading libaries 
import requests
import json
import os
import subprocess
import sys
import base64
import hashlib
from urllib.parse import urlparse
import subprocess
import json
import time
import os
import glob


RSA_SIZE = 2048


DEFAULT_CURVE = "secp384r1"
FAST_CURVE = "secp256r1"
SUPPORTED_CURVES = [DEFAULT_CURVE, FAST_CURVE]

DIGEST_SHA384 = "sha384"
DIGEST_SHA256 = "sha256"

RSA_SIZE = 2048
# CCF network node
server="https://127.0.0.1:8000"

num_users = 4

In [184]:
# Getting Network metrices
url = server + "/app/api/metrics"

try:
    response = requests.get(url, verify='service_cert.pem')

    print("Status Code:", response.status_code)
    print("\nResponse Headers:")
    for header, value in response.headers.items():
        print(f"{header}: {value}")

    print("\nResponse Body:")
    try:
        # Attempt to parse JSON and print it in an indented format
        response_json = response.json()
        print(json.dumps(response_json, indent=4))
    except ValueError:
        # If response is not JSON, print as plain text
        print(response.text)

except requests.exceptions.RequestException as e:
    print("Error making request:", e)


Status Code: 200

Response Headers:
content-length: 176
content-type: application/json
x-ms-ccf-transaction-id: 2.12

Response Body:
{
    "metrics": [
        {
            "calls": 1,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "api/metrics",
            "retries": 0
        },
        {
            "calls": 2,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "commit",
            "retries": 0
        }
    ]
}


In [185]:
import os
import glob

def delete_user_and_member_files(num_users, num_members):
    for i in range(num_users):
        # User file names based on user index
        user_files = [f"user{i}_cert.pem", f"user{i}_privk.pem", f"user{i}_enc_privk.pem", f"user{i}_enc_pubk.pem"]

        # Delete user files if they exist
        for file in user_files:
            if os.path.exists(file):
                os.remove(file)
                print(f"Deleted {file}")
            else:
                print(f"{file} does not exist")

        # Delete all JSON files related to the user
        for json_file in glob.glob(f"set_user{i}*.json"):
            os.remove(json_file)
            print(f"Deleted {json_file}")

    for j in range(1, num_members + 1):
        # Member file names based on member index
        member_files = [f"member{j}_cert.pem", f"member{j}_privk.pem", f"member{j}_enc_privk.pem", f"member{j}_enc_pubk.pem"]

        # Delete member files if they exist
        for file in member_files:
            if os.path.exists(file):
                os.remove(file)
                print(f"Deleted {file}")
            else:
                print(f"{file} does not exist")

        # Delete all JSON files related to the member
        for json_file in glob.glob(f"set_member{j}*.json"):
            os.remove(json_file)
            print(f"Deleted {json_file}")

# Example usage
delete_user_and_member_files(num_users=3, num_members=2)


user0_cert.pem does not exist
user0_privk.pem does not exist
user0_enc_privk.pem does not exist
user0_enc_pubk.pem does not exist
user1_cert.pem does not exist
user1_privk.pem does not exist
user1_enc_privk.pem does not exist
user1_enc_pubk.pem does not exist
user2_cert.pem does not exist
user2_privk.pem does not exist
user2_enc_privk.pem does not exist
user2_enc_pubk.pem does not exist
member1_cert.pem does not exist
member1_privk.pem does not exist
member1_enc_privk.pem does not exist
member1_enc_pubk.pem does not exist
member2_cert.pem does not exist
member2_privk.pem does not exist
member2_enc_privk.pem does not exist
member2_enc_pubk.pem does not exist


In [187]:
# Keygenrator function for public and private key with ballat request file


def generate_keys(name, curve=DEFAULT_CURVE, generate_encryption_key=False):
    if not name:
        print("Error: The name of the participant should be specified (e.g. member0 or user1)")
        sys.exit(1)

    if curve not in SUPPORTED_CURVES:
        print(f"{curve} curve is not in {SUPPORTED_CURVES}")
        sys.exit(1)

    digest = DIGEST_SHA384 if curve == DEFAULT_CURVE else DIGEST_SHA256

    cert = f"{name}_cert.pem"
    privk = f"{name}_privk.pem"

    print(f"-- Generating identity private key and certificate for participant \"{name}\"...")
    print(f"Identity curve: {curve}")

    subprocess.run(["openssl", "ecparam", "-out", privk, "-name", curve, "-genkey"], check=True)
    subprocess.run(["openssl", "req", "-new", "-key", privk, "-x509", "-nodes", "-days", "365", "-out", cert, f"-{digest}", "-subj", f"/CN={name}"], check=True)

    print(f"Identity private key generated at: {privk}")
    print(f"Identity certificate generated at: {cert}")

    enc_priv, enc_pub = None, None
    if generate_encryption_key:
        print(f"-- Generating RSA encryption key pair for participant \"{name}\"...")

        enc_priv = f"{name}_enc_privk.pem"
        enc_pub = f"{name}_enc_pubk.pem"

        subprocess.run(["openssl", "genrsa", "-out", enc_priv, str(RSA_SIZE)], check=True)
        subprocess.run(["openssl", "rsa", "-in", enc_priv, "-pubout", "-out", enc_pub], check=True)

        print(f"Encryption private key generated at: {enc_priv}")
        print(f"Encryption public key generated at: {enc_pub}")

    return cert, enc_pub





def create_certificatefor_member(cert_name):
    cert_file = f"{cert_name}_cert.pem"
    set_member_file = f"set_{cert_name}.json"
    
    # Call the generate_keys function (make sure to include it in your script)
    cert_content, pub_key_content = generate_keys(cert_name)

    # Read the certificate file and format it
    with open(cert_file, 'r') as file:
        cert_content = file.read().replace('\n', '\n')

    # Create the JSON content
    user_json = {
        "actions": [
            {
                "name": "set_member",
                "args": {
                    "cert": cert_content,
                    "encryption_pub_key": pub_key_content
                }
            }
        ]
    }

    # Write the JSON to a file
    with open(set_member_file, 'w') as file:
        json.dump(user_json, file, indent=2)
    print(f"JSON file created at: {set_member_file}")
    


def send_secure_request(url, request_data_path, signing_privk, signing_cert, command="post"):
    def read_request_data(request_path):
        if request_path.startswith('@'):
            with open(request_path[1:], 'r') as file:
                return file.read()
        else:
            return request_path

    def calculate_digest(data):
        sha256_hash = hashlib.sha256()
        sha256_hash.update(data.encode('utf-8'))
        return base64.b64encode(sha256_hash.digest()).decode()

    def create_signature(string_to_sign, priv_key_path):
        process = subprocess.Popen(
            ['openssl', 'dgst', '-sha384', '-sign', priv_key_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        signature, _ = process.communicate(string_to_sign.encode())
        return base64.b64encode(signature).decode().replace('\n', '')

    def prepare_string_to_sign(method, url, digest, data_length):
        parsed_url = urlparse(url)
        path = parsed_url.path
        return f"(request-target): {method.lower()} {path}\ndigest: SHA-256={digest}\ncontent-length: {data_length}"

    def get_cert_key_id(cert_path):
        proc = subprocess.Popen(
            ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, _ = proc.communicate()
        fingerprint = output.decode().split('=')[1].replace(':', '').lower().replace('\n', '')
        return fingerprint

    request_data = read_request_data(request_data_path)
    request_digest = calculate_digest(request_data)
    string_to_sign = prepare_string_to_sign(command, url, request_digest, str(len(request_data)))
    signature = create_signature(string_to_sign, signing_privk)
    key_id = get_cert_key_id(signing_cert)

    headers = {
        "Digest": f"SHA-256={request_digest}",
        "Authorization": f'Signature keyId="{key_id}",algorithm="hs2019",headers="(request-target) digest content-length",signature="{signature}"'
    }

    response = requests.post(url, data=request_data, headers=headers, verify='service_cert.pem')
    return response.status_code, response.text




# sending voting to add  the user based on proposal id 

def send_ballot_request(server_url, proposal_id, data_file_path, signing_privk_path, signing_cert_path):
    def read_request_data(file_path):
        with open(file_path, 'r') as file:
            return file.read()

    def calculate_digest(data):
        sha256_hash = hashlib.sha256()
        sha256_hash.update(data.encode('utf-8'))
        return base64.b64encode(sha256_hash.digest()).decode()

    def create_signature(string_to_sign, priv_key_path):
        process = subprocess.Popen(
            ['openssl', 'dgst', '-sha384', '-sign', priv_key_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        signature, _ = process.communicate(string_to_sign.encode())
        return base64.b64encode(signature).decode().replace('\n', '')

    def prepare_string_to_sign(url, digest, data_length):
        parsed_url = urlparse(url)
        path = parsed_url.path
        return f"(request-target): post {path}\ndigest: SHA-256={digest}\ncontent-length: {data_length}"

    def get_cert_key_id(cert_path):
        proc = subprocess.Popen(
            ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, _ = proc.communicate()
        fingerprint = output.decode().split('=')[1].replace(':', '').lower().replace('\n', '')
        return fingerprint

    url = f"{server_url}/gov/proposals/{proposal_id}/ballots"
    request_data = read_request_data(data_file_path)
    request_digest = calculate_digest(request_data)
    string_to_sign = prepare_string_to_sign(url, request_digest, str(len(request_data)))
    signature = create_signature(string_to_sign, signing_privk_path)
    key_id = get_cert_key_id(signing_cert_path)

    headers = {
        "Digest": f"SHA-256={request_digest}",
        "Authorization": f'Signature keyId="{key_id}",algorithm="hs2019",headers="(request-target) digest content-length",signature="{signature}"',
        "Content-Type": "application/json"
    }

    response = requests.post(url, data=request_data, headers=headers, verify='service_cert.pem')
    return response.status_code, response.json()





# creation of user id based on certifacte 
def get_certificate_fingerprint(cert_path):
    process = subprocess.Popen(
        ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    output, _ = process.communicate()
    fingerprint = output.decode().split('=')[1].replace(':', '').lower().strip()
    return fingerprint



import json

def create_certificate(cert_name, entity_type):
    cert_file = f"{cert_name}_cert.pem"
    pub_key_file = f"{cert_name}_enc_pubk.pem"  # Public key file name
    set_file = f"set_{cert_name}.json"
    
    # Adjust the generate_keys function call based on entity type
    if entity_type == "member":
        cert_content, pub_key_content = generate_keys(cert_name, generate_encryption_key=True)
    elif entity_type == "user":
        cert_content = generate_keys(cert_name, generate_encryption_key=False)[0]
    else:
        raise ValueError("entity_type must be 'user' or 'member'")

    # Read the certificate file and format it
    with open(cert_file, 'r') as file:
        cert_content = file.read()

    # Create the JSON content
    action_args = {"cert": cert_content}
    if entity_type == "member":
        # Read the public key file and format it
        with open(pub_key_file, 'r') as file:
            pub_key_content = file.read()
        action_args["encryption_pub_key"] = pub_key_content

    user_json = {
        "actions": [
            {
                "name": f"set_{entity_type}",
                "args": action_args
            }
        ]
    }

    # Write the JSON to a file
    with open(set_file, 'w') as file:
        json.dump(user_json, file, indent=2)
    print(f"JSON file created at: {set_file}")

# Replace generate_keys call with the actual implementation




In [188]:
# Add member 
# Example usage
# Create member1
create_certificate("member1", "member")



-- Generating identity private key and certificate for participant "member1"...
Identity curve: secp384r1
Identity private key generated at: member1_privk.pem
Identity certificate generated at: member1_cert.pem
-- Generating RSA encryption key pair for participant "member1"...
Encryption private key generated at: member1_enc_privk.pem
Encryption public key generated at: member1_enc_pubk.pem
JSON file created at: set_member1.json


writing RSA key


In [193]:
# Create member2
create_certificate("member2", "member")


-- Generating identity private key and certificate for participant "member2"...
Identity curve: secp384r1
Identity private key generated at: member2_privk.pem
Identity certificate generated at: member2_cert.pem
-- Generating RSA encryption key pair for participant "member2"...
Encryption private key generated at: member2_enc_privk.pem
Encryption public key generated at: member2_enc_pubk.pem
JSON file created at: set_member2.json


writing RSA key


In [194]:

# Adding new two member and then activationg

member1 = "member1"
member2 = "member2"

# Send request to add member1
status_code, response_json = send_secure_request(
    server + "/gov/proposals",
    f"@set_{member2}.json",
    "member0_privk.pem",
    "member0_cert.pem",
    "post"
)
print("Status code:", status_code, "Response:", response_json)
response=json.loads(response_json)
# Check if the request was successful and then proceed to vote
if status_code == 200 and response["state"]=="open" :
    # Extract the proposal ID from response_json, replace 'proposal_id_field' with the actual field name
    proposal_id=response['proposal_id']

    # Send a vote for the proposal
    vote_status, response_json = send_ballot_request(
        server,
        proposal_id,
        "vote_accept.json",
        "member0_privk.pem",
        "member0_cert.pem"
    )
    print("Voting status:", vote_status, "Response:", response_json)
elif status_code==200 and response["state"]=="Accepted":
    print("propsal is already accpeted")
else:
    print("Failed to add member, status code:", status_code)
  


Status code: 200 Response: {"ballot_count":0,"proposal_id":"e95a83e7dda04bfa03037f743a845003e364bb6b21cc847dc2a590c95013bd38","proposer_id":"2c860575e24c88bc239f69833051401b5bac15016ad60f80d9b7e4b63f5c95f0","state":"Accepted","votes":{}}
propsal is already accpeted


In [212]:
import requests
import subprocess
import hashlib
import base64
from urllib.parse import urlparse
import json

def send_ack_request(server_url, state_digest, signing_privk_path, signing_cert_path):
    def create_signature(string_to_sign, priv_key_path):
        process = subprocess.Popen(
            ['openssl', 'dgst', '-sha384', '-sign', priv_key_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        signature, _ = process.communicate(string_to_sign.encode())
        return base64.b64encode(signature).decode().replace('\n', '')

    def prepare_string_to_sign(url, digest, data_length):
        parsed_url = urlparse(url)
        path = parsed_url.path
        return f"(request-target): post {path}\ndigest: SHA-256={digest}\ncontent-length: {data_length}"

    def get_cert_key_id(cert_path):
        proc = subprocess.Popen(
            ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, _ = proc.communicate()
        fingerprint = output.decode().split('=')[1].replace(':', '').lower().replace('\n', '')
        return fingerprint

    def calculate_digest(data):
        sha256_hash = hashlib.sha256()
        sha256_hash.update(data.encode('utf-8'))
        return base64.b64encode(sha256_hash.digest()).decode()

    url = f"{server_url}/gov/ack"
    request_data = json.dumps({"state_digest": state_digest})
    request_digest = calculate_digest(request_data)
    string_to_sign = prepare_string_to_sign(url, request_digest, str(len(request_data)))
    signature = create_signature(string_to_sign, signing_privk_path)
    key_id = get_cert_key_id(signing_cert_path)

    headers = {
        "Digest": f"SHA-256={request_digest}",
        "Authorization": f'Signature keyId="{key_id}",algorithm="hs2019",headers="(request-target) digest content-length",signature="{signature}"',
        "Content-Type": "application/json"
    }

    response = requests.post(url, data=request_data, headers=headers, verify='service_cert.pem')
    return response.status_code, response.json() if response.status_code != 204 else {}





In [213]:
import requests
import json

def create_and_save_state_digest(server, cert_path, priv_key_path, output_file):
    update_state_digest_url = server + "/gov/ack/update_state_digest"

    try:
        response = requests.post(update_state_digest_url, cert=(cert_path, priv_key_path), verify="service_cert.pem")

        # Check if the request was successful
        if response.status_code == 200:
            # Parse response to JSON
            response_json = response.json()
            print("Response:", response_json)

            # Write the response to a JSON file
            with open(output_file, 'w') as file:
                json.dump(response_json, file, indent=4)
            print(f"State digest saved to {output_file}")
        else:
            print(f"Request failed with status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")


cert = "member2_cert.pem"
private_key = "member2_privk.pem"
output_file = "member2_request.json"

create_and_save_state_digest(server, cert, private_key, output_file)


Response: {'state_digest': 'ae3d3ef06e5460820379f5147c46237f60bc7b4cdaae160571ab99037535eacd'}
State digest saved to member2_request.json


In [215]:
import json

def read_state_digest(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        return data['state_digest']

# Read the state digest from request.json
state_digest = read_state_digest("member2_request.json")

# Now pass the actual state digest to the function
response_code, response_data = send_ack_request(server, state_digest, "member2_privk.pem", "member2_cert.pem")
print("response", response_code, "responsdata", response_data)

response 400 responsdata {'error': {'code': 'StateDigestMismatch', 'message': 'Submitted state digest is not valid.'}}


In [ ]:
member2=get_certificate_fingerprint("member2_cert.pem")

In [239]:
urlr=f"{server}/gov/service/members/{member2}"
print(urlr)
try:
    response = requests.get(urlr, verify='service_cert.pem',cert=("member2_cert.pem","member2_privk.pem"))

    print("Status Code:", response.status_code)
    print("\nResponse Headers:")
    for header, value in response.headers.items():
        print(f"{header}: {value}")

    print("\nResponse Body:")
    try:
        # Attempt to parse JSON and print it in an indented format
        response_json = response.json()
        print(json.dumps(response_json, indent=4))
    except ValueError:
        # If response is not JSON, print as plain text
        print(response.text)

except requests.exceptions.RequestException as e:
    print("Error making request:", e)


https://127.0.0.1:8000/gov/service/members/2a3c15070488c2914ce7302bb818728f998a72907a44866631e056052528acf1
Status Code: (404, '{"error":{"code":"ResourceNotFound","message":"Unknown path: /service/members/2a3c15070488c2914ce7302bb818728f998a72907a44866631e056052528acf1."}}')

Response Headers:


In [229]:
def main_process(num_initial_users):
    # Generate keys and certificates for all users
    for i in range(num_initial_users):
        create_certificate(f"user{i}","user")

    # Submit proposal and voting for each user
    for i in range(num_initial_users):
        user_name = f"user{i}"
        user_cert_path = f"{user_name}_cert.pem"
        user_key_path = f"{user_name}_privk.pem"

        # Member0 submits the proposal
        status_code, response_json = send_secure_request(server + "/gov/proposals", f"@set_{user_name}.json", "member1_privk.pem", "member1_cert.pem", "post")
        print("proposal_id",json.loads(response_json)['proposal_id'])
        proposal_id=json.loads(response_json)['proposal_id']

        # Other members vote
        for j in range(1, 3):  # Assuming 2 other members (member1 and member2)
            vote_status, response_json = send_ballot_request(server, proposal_id, "vote_accept.json", f"member{j}_privk.pem", f"member{j}_cert.pem")
            print("vote_status",vote_status,"response_json",response_json)
            if vote_status != 200:
                print(f"Voting failed for {user_name} by member{j}")
                continue
            time.sleep(1)  # Delay for processing

        # Create account for the user
        user_id=get_certificate_fingerprint(user_cert_path)
        print("Actual id",user_id) 
   
      
       


# Run the script for a specified number of users
main_process(2)

-- Generating identity private key and certificate for participant "user0"...
Identity curve: secp384r1
Identity private key generated at: user0_privk.pem
Identity certificate generated at: user0_cert.pem
JSON file created at: set_user0.json
-- Generating identity private key and certificate for participant "user1"...
Identity curve: secp384r1
Identity private key generated at: user1_privk.pem
Identity certificate generated at: user1_cert.pem
JSON file created at: set_user1.json
proposal_id e66e6452c9825471c88cc8f379add1832916acab45f11c6297de4ef32a04dc83
vote_status 400 response_json {'error': {'code': 'ProposalNotOpen', 'message': 'Proposal e66e6452c9825471c88cc8f379add1832916acab45f11c6297de4ef32a04dc83 is currently in state accepted - only open proposals can receive votes.'}}
Voting failed for user0 by member1
vote_status 400 response_json {'error': {'code': 'ProposalNotOpen', 'message': 'Proposal e66e6452c9825471c88cc8f379add1832916acab45f11c6297de4ef32a04dc83 is currently in state